In [ ]:
import torch
from torch.utils.data import DataLoader

from src.dataset import COCODataset
from src.trainer import Trainer
from src.utils import load_config, download_coco_data, seed_everything
from src.visuals import Segmentor

print("Pytorch version:", torch.__version__)
print("CUDA enabled:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("Device:", torch.cuda.get_device_name())

In [ ]:
CONFIG = load_config()
seed_everything(CONFIG.seed)
segmentor = Segmentor()
CONFIG

In [ ]:
data = download_coco_data(config=CONFIG, split="train", max_samples=2000)
print(data)

# import fiftyone
# fiftyone.launch_app(data)

In [ ]:
train_dataset = COCODataset(data[:1600])
train_dataset.summary()
image, masks = train_dataset[0]
segmentor.visualize_mask(image, masks)

In [ ]:
val_dataset = COCODataset(data[1600:1800])
val_dataset.summary()
image, masks = val_dataset[10]
segmentor.visualize_mask(image, masks)

In [ ]:
test_dataset = COCODataset(data[1800:])
test_dataset.summary()
image, masks = test_dataset[6]
segmentor.visualize_mask(image, masks)

In [ ]:
train_dl = DataLoader(train_dataset, batch_size=CONFIG.batch_size, pin_memory=True)
val_dl = DataLoader(val_dataset, batch_size=CONFIG.batch_size, pin_memory=True)
test_dl = DataLoader(test_dataset, batch_size=CONFIG.batch_size, pin_memory=True)

In [ ]:
trainer = Trainer(dataloaders=(train_dl, val_dl))

In [ ]:
trainer.fit()

In [ ]:
for dataloader in (train_dl, val_dl, test_dl):
    losses, iou, m_iou = trainer.eval(dataloader)
    trainer.log_losses(losses, phase="eval")
    trainer.log_iou(iou, m_iou)
    print("====================================")